In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


24/02/14 02:08:47 WARN Utils: Your hostname, Nicolass-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.20.10.2 instead (on interface en0)
24/02/14 02:08:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/14 02:08:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import os
local_path = os.getcwd()
local_path

'/Users/nicolascorchuelo/Documents/projects/portfolio/languages_programming/python/pyspark/standalone'

24/02/14 02:09:05 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
from pyspark.sql.functions import regexp_replace

file_location = "/source/FILE_NAME.CSV.gz"

df3 = spark \
    .read.format("csv") \
    .option("compression", "gzip") \
    .option("sep", ";") \
    .option("header", True) \
    .load(local_path + file_location) \

nuevos_nombres = ["fechaproceso","name","decimal_1","decimal_2"]

df3_con_nuevos_nombres = df3.toDF(*nuevos_nombres)   

columnas_a_reemplazar = ['decimal_1', 'decimal_2']

# Apply regexp_replace in a single call to withColumn
for col in columnas_a_reemplazar:
    df3_con_nuevos_nombres = df3_con_nuevos_nombres.withColumn(col, regexp_replace(col, ',', '.'))

df3_con_nuevos_nombres.show()

+-------------------+-----------------+---------+---------+
|       fechaproceso|             name|decimal_1|decimal_2|
+-------------------+-----------------+---------+---------+
|01-12-2023 00:00:00|NICOLAS CORCHUELO|  0.00737|  0.16454|
+-------------------+-----------------+---------+---------+



24/02/14 02:09:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: dateprocess, name,decimal_1, decimal_2, decimal_2
 Schema: dateprocess, name,decimal_1, decimal_22, decimal_23
Expected: decimal_22 but found: decimal_2
CSV file: file:///Users/nicolascorchuelo/Documents/projects/portfolio/languages_programming/python/pyspark/standalone/source/FILE_NAME.CSV.gz


In [5]:
# Create a view or table

temp_table_name = "file_table"

df3_con_nuevos_nombres.createOrReplaceTempView(temp_table_name)